In [54]:
import os
import cv2
import argparse
import shutil
import numpy as np
from lxml import etree
from tqdm import tqdm

### Parse CVAT .xml annotations

In [80]:
root = etree.parse('annotations.xml').getroot()
images = []

for img in root.iterfind('image'):
    name = img.attrib['name']
    for item in img:
        label = item.attrib['label']
        
    images.append((name, label))

In [82]:
df = pd.DataFrame(images, columns=['file_name', 'category'])
df

,file_name,category
0,2020-01-07_17_22_46-81_color.png,plastic
1,2020-01-07_17_22_39-46_color.png,plastic
2,2020-01-07_17_22_37-17_color.png,plastic
3,2020-01-07_17_22_34-34_color.png,plastic
4,2020-01-07_17_22_31-77_color.png,plastic
...,...,...
2470,2019-09-19_16_20_00-38_color.png,plastic
2471,2019-09-19_16_19_55-63_color.png,plastic
2472,2019-09-19_16_19_49-57_color.png,carton
2473,2019-09-19_16_19_44-93_color.png,carton


### Replace the categories in the annotations .json files

In [100]:
def get_new_categories(image_id, images_df, categories_df):
    filename = images_df[images_df['id'] == image_id]['file_name'].values[0]
    label = categories_df[categories_df['file_name'] == filename]['category'].values[0]
    
    return label

In [101]:
annotation_files = ['fixed_train.json', 'fixed_val.json', 'fixed_test.json']
replacements = {
    'plastic': 1,
    'carton': 2,
    'glass': 3,
    'organic': 4,
    'rest': 5,
    'other': 6,
    'dangerous': 7
}

for file in annotation_files:
    with open(file, 'r') as input_file:
        data = json.load(input_file)
        
    temp_annotations_df = pd.DataFrame(data['annotations'])
    temp_images_df = pd.DataFrame(data['images'])
    
    temp_annotations_df['category_id'] = temp_annotations_df['image_id'].apply(get_new_categories, args=(temp_images_df, df))
    temp_annotations_df['category_id'] = temp_annotations_df['category_id'].replace(replacements)
    
    annotations_dict = temp_annotations_df.to_dict(orient='records')
    data['annotations'] = annotations_dict
    
    with open(f'cat_{file}', 'w') as outfile:
        json.dump(data, outfile)